##                                          Author: Jinzhou Yao
##                                 Description: Risk management HW2

### Import python modules

In [1]:
from yahoofinancials import YahooFinancials
import pandas as pd
from pandas_datareader import data
import numpy as np
from math import sqrt, pi, log, e, erf
from scipy.stats import norm
from scipy.stats import multivariate_normal
from scipy.optimize import fsolve

### Collect one year daily stock prices for 10 companies. 

In [2]:
tickers = ['MSFT', 'AAPL', 'AMZN', 'VZ','JPM','TSLA','FB','IBM','GOOG','DELL']
start_date = '2018-04-01'
end_date = '2019-04-01'
panel_data = data.DataReader(tickers, 'yahoo', start_date, end_date)
adjclose = panel_data['Adj Close']
adjclose = adjclose.fillna(method='ffill')
print(adjclose)

Symbols           AAPL         AMZN       DELL          FB         GOOG  \
Date                                                                      
2018-04-02  164.180008  1371.989990  40.177189  155.389999  1006.469971   
2018-04-03  165.864349  1392.050049  40.321152  156.110001  1013.409973   
2018-04-04  169.036072  1410.569946  40.404209  155.100006  1025.140015   
2018-04-05  170.208206  1451.750000  40.315613  159.339996  1027.810059   
2018-04-06  165.854523  1405.229980  39.473976  157.199997  1007.039978   
2018-04-09  167.499466  1406.079956  39.811737  157.929993  1015.450012   
2018-04-10  170.651459  1436.219971  40.304539  165.039993  1031.640015   
2018-04-11  169.853622  1427.050049  39.828350  166.320007  1019.969971   
2018-04-12  171.528107  1448.500000  40.243633  163.869995  1032.510010   
2018-04-13  172.109268  1430.790039  40.365448  164.520004  1029.270020   
2018-04-16  173.182922  1441.500000  41.168327  164.830002  1037.979980   
2018-04-17  175.566620  1

### Compute daily returns, and then mean and standard deviation. 

In [3]:
ret =adjclose.diff()/adjclose.shift(1)
ret.drop(ret.index[0], inplace=True)
mu = ret.apply(np.average, axis=0)
sigma = ret.apply(np.std, axis=0)
print("Return:\n",ret)
print("Mean:\n",mu)
print("Standard deviation:\n",sigma)

Return:
 Symbols         AAPL      AMZN      DELL        FB      GOOG       IBM  \
Date                                                                     
2018-04-03  0.010259  0.014621  0.003583  0.004634  0.006895 -0.001466   
2018-04-04  0.019122  0.013304  0.002060 -0.006470  0.011575  0.028495   
2018-04-05  0.006934  0.029194 -0.002193  0.027337  0.002605 -0.000584   
2018-04-06 -0.025579 -0.032044 -0.020876 -0.013430 -0.020208 -0.022463   
2018-04-09  0.009918  0.000605  0.008557  0.004644  0.008351  0.014080   
2018-04-10  0.018818  0.021435  0.012378  0.045020  0.015944  0.017683   
2018-04-11 -0.004675 -0.006385 -0.011815  0.007756 -0.011312 -0.000193   
2018-04-12  0.009858  0.015031  0.010427 -0.014731  0.012295  0.017443   
2018-04-13  0.003388 -0.012226  0.003027  0.003967 -0.003138 -0.008604   
2018-04-16  0.006238  0.007485  0.019890  0.001884  0.008462  0.007530   
2018-04-17  0.013764  0.043240  0.013988  0.023236  0.034856  0.019127   
2018-04-18 -0.002244  0.01596

### Collect current and long-term liabilities of each firm

In [5]:
yahoo_financials = YahooFinancials(tickers)
balance_sheet_data = yahoo_financials.get_financial_stmts('annual', 'balance')
balance_sheet_all = balance_sheet_data['balanceSheetHistory']
total_liab = {}
short_liab = {}
equity = {}
for key, value in balance_sheet_all.items():
    a = value[0]
    for k1, v1 in a.items():
        total_liab[key] = v1['totalLiab']
        short_liab[key] = v1['totalCurrentLiabilities']
        equity[key] = v1['totalStockholderEquity']
total = pd.Series(total_liab)
short = pd.Series(short_liab)
long=total-short
equity = pd.Series(equity)
print("Long-term liabilities:\n",long)
print("current liabilities:\n",short)

Long-term liabilities:
 MSFT    117642000000
AAPL    141712000000
AMZN     50708000000
VZ      172189000000
JPM     270707000000
TSLA     13433874000
FB        6190000000
IBM      68225000000
GOOG     20544000000
DELL     67790000000
dtype: int64
current liabilities:
 MSFT      58488000000
AAPL     116866000000
AMZN      68391000000
VZ        37930000000
JPM     2095310000000
TSLA       9992136000
FB         7017000000
IBM       38227000000
GOOG      34620000000
DELL      44972000000
dtype: int64


### retrieve the default probability of each firm

In [9]:
stocks = pd.DataFrame({'mu':mu,'sigma':sigma, 'total_liab':total, 'short_liab': short_liab, 'total_equity': equity})
stocks['long_liab'] = stocks['total_liab']-stocks['short_liab']
T1 = 1.0
T2 = 2.0
K1 = np.array(stocks['short_liab'],dtype = float)
K2 = np.array(stocks['long_liab'],dtype = float)
def eq1(variable, *args):
    A_bar1, x1, x2= variable
    mu, sigma, K1, K2 = args[0], args[1],args[2],args[3]
    
    
    return (A_bar1 * norm.cdf(x1) -np.exp(-mu*(T2-T1))*K2* norm.cdf(x2) -K1, 
            (np.log(A_bar1) -np.log(K2) +(mu+sigma**2/2)*(T2-T1)) / (sigma*np.sqrt(T2-T1)) -x1,
            x1-(sigma*np.sqrt(T2-T1)) -x2  )
A_bar1=np.zeros(10)
x1=np.zeros(10)
x2=np.zeros(10)
for i in range(10):
    A_bar1[i], x1[i],x2[i] =  fsolve(eq1,(1000000, 0.002, -100),(mu[i],sigma[i],K1[i],K2[i]))

A_bar2 = K2.copy()
rho = np.sqrt(T1/T2)
mean = np.array([0,0])
covariance = np.array([[1, rho],[rho, 1]])
dist = multivariate_normal(mean=mean, cov=covariance)

#Geske model
def eq2(variable, *args):
    A, sigmaA, h1a,h1b, h2a,h2b= variable
    mu, sigma, K1, K2, equity, A_bar1, A_bar2 = args[0], args[1],args[2],args[3],args[4],args[5],args[6]
    h_a = np.array([h1a,h2a])
    h_b = np.array([h1b,h2b])
    return (A * dist.cdf(h_a) -np.exp(-mu*(T2))*K2* dist.cdf(h_b) -np.exp(-mu*(T1))*K1* norm.cdf(h1b) - equity, 
            sigmaA*dist.cdf(h_a)*A/equity - sigma,
            
            (np.log(A) -np.log(A_bar1) +(mu+sigma**2/2)*(T1)) / (sigma*np.sqrt(T1)) -h1a,
            h1a-(sigma*np.sqrt(T1)) -h1b,
            
            (np.log(A) -np.log(A_bar2) +(mu+sigma**2/2)*(T2)) / (sigma*np.sqrt(T2)) -h2a,
            h2a-(sigma*np.sqrt(T2)) -h2b,
            )
A=np.zeros(10)
sigmaA=np.zeros(10)
h1a=np.zeros(10)
h1b=np.zeros(10)
h2a=np.zeros(10)
h2b=np.zeros(10)
for i in range(10):
    A[i], sigmaA[i], h1a[i],h1b[i], h2a[i],h2b[i] =  fsolve(eq2,
     (1000000, 0.1, 1, 10 ,30, 100),(mu[i],sigma[i],K1[i],K2[i],equity[i],A_bar1[i],A_bar2[i]))
S1 = norm.cdf(h1b)
S2 = np.array([dist.cdf([h1b[i],h2b[i]]) for i in range(10)])
D1 = 1-S1
D2 = 1-S2/S1
print("Default probability:\n",D1,"\n",D2,"\n")

Default probability:
 [0.         0.         0.         0.         0.         0.
 0.00323075 0.         0.         0.        ] 
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 



### Author Jinzhou Yao
### Mini HW4

In [2]:
class EquityOption:     
    RiskfreeRate = 0.05
    Vol = 0.3       
    def __init__ (self, *args):
        self.CallFlag = int(args[0])
        self.Spot = float(args[1])
        self.Strike = float(args[2])
        self.Maturity = float(args[3])
        self.DividendYield =  float(args[4])
        
    def __imul__(self, stock_split):
        self.Spot = self.Spot * stock_split 
        self.Strike = self.Strike * stock_split
        return self
    
    @staticmethod 
    def N(x): 
        return (0.5*(1+ erf(x / sqrt(2.0))))
    def d1(self):
        return float((log(self.Spot/self.Strike) + (self.RiskfreeRate - self.DividendYield + (self.Vol**2) / 2  ) * self.Maturity) / (self.Vol * sqrt(self.Maturity)));
    def d2(self): 
        return float(self.d1() - self.Vol*sqrt(self.Maturity))
    def BS_Option(self, v=Vol):
        self.Vol = v #essential step
        dfq = e**(-self.DividendYield * self.Maturity)
        dfr = e**(-self.RiskfreeRate * self.Maturity)
        if self.CallFlag: 
            return self.Spot * dfq* self.N(self.d1()) - self.Strike*dfr * self.N(self.d2())
        else: 
            return self.Strike*dfr * self.N(-self.d2()) - self.Spot * dfq* self.N(-self.d1())

In [4]:
sigma,S0,rf,T,d,CallFlag = 0.3,100,0.05,1,0,1
K0,K1,K2,K3,K4,K5 = 0,1250*0.03,1250*0.07,1250*0.1,1250*0.3,1250*1
C0,C1,C2,C3,C4,C5 = 100,EquityOption(CallFlag, S0, K1, T, d).BS_Option(),EquityOption(CallFlag, S0, K2, T, d).BS_Option(),EquityOption(CallFlag, S0, K3, T, d).BS_Option(),EquityOption(CallFlag, S0, K4, T, d).BS_Option(),EquityOption(CallFlag, S0, K5, T, d).BS_Option() 
print('The expected losses of equity tranche is ', C1-C0,'millions')
print('The expected losses of mezzanine tranche1 is ', C2-C1,'millions')
print('The expected losses of mezzanine  tranche2 is ', C3-C2,'millions')
print('The expected losses of senior  tranche is ', C4- C3,'millions')
print('The expected losses of super-senior tranche is ', C5-C4,'millions')


The expected losses of equity tranche is  -35.66976819875484 millions
The expected losses of mezzanine tranche1 is  -43.06153312847525 millions
The expected losses of mezzanine  tranche2 is  -15.577174485188284 millions
The expected losses of senior  tranche is  -5.691388992284564 millions
The expected losses of super-senior tranche is  -0.0001351952970400196 millions


### plt.GridSpec(): Flexible Subplots Arrangements